In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from IPython.display import display, clear_output

#sudoku problem genertor
# Fill a 3x3 diagonal box
def fill_box(board, row, col):
    nums = np.random.choice(np.arange(1, 10), size=9, replace=False).reshape(3,3)
    board[row:row+3, col:col+3] = nums

# Step 1: Fill diagonal boxes
def fill_diagonal(board):
    for i in range(0, 9, 3):
        fill_box(board, i, i)

# Step 3: Remove numbers
def remove_cells(board, count):
    flat_idx = np.random.choice(81, size=count, replace=False)
    for idx in flat_idx:
        r = idx // 9
        c = idx % 9
        board[r, c] = 0

# Full generator
def generate_sudoku(remove_count=45):
    board = np.zeros((9,9), dtype=int)

    fill_diagonal(board) 
    remove_cells(board, remove_count) 

    return board

# Sample Hard Puzzle
board = generate_sudoku()



In [ ]:
# Visualize the board    

# Setup the figure size
fig, ax = plt.subplots(figsize=(6, 6))

def update_plot(board):
        ax.clear()
        # Create a heatmap-style grid
        ax.imshow(board, cmap='Blues', vmin=0, vmax=9)
        
        # Draw the grid lines
        for i in range(10):
            lw = 3 if i % 3 == 0 else 0.5
            ax.axhline(i - 0.5, color='black', lw=lw)
            ax.axvline(i - 0.5, color='black', lw=lw)
            
        # Draw the numbers
        for (r, c), val in np.ndenumerate(board):
            if val != 0:
                ax.text(c, r, str(int(val)), ha='center', va='center', fontsize=16)
        
        ax.set_xticks([]); ax.set_yticks([])
        ax.set_title("Solving...", fontsize=14)
        
        # KEY CHANGES FOR JUPYTER:
        clear_output(wait=True) # Clears the cell output
        display(fig)            # Forces the figure to display
        time.sleep(0.025)        # Control speed (0.05s per frame)

In [ ]:


def solve_visual_notebook(grid):
    
    update_plot(board)
    

    # Standard Bitmask Logic
    rows, cols, boxes = np.zeros(9, int), np.zeros(9, int), np.zeros(9, int)
    get_box = lambda r, c: (r // 3) * 3 + (c // 3)

    # Pre-fill constraints
    for r, c in np.ndindex(9, 9):
        if grid[r, c] > 0:
            mask = 1 << (grid[r, c] - 1)
            rows[r] |= mask; cols[c] |= mask; boxes[get_box(r, c)] |= mask

    def backtrack():
        # MRV Heuristic
        best_cell, min_choices = None, 10
        for r, c in np.ndindex(9, 9):
            if grid[r, c] == 0:
                allowed = 0x1FF & ~(rows[r] | cols[c] | boxes[get_box(r, c)])
                choices = bin(allowed).count('1')
                if choices < min_choices:
                    min_choices, best_cell, best_allowed = choices, (r, c), allowed
        
        if not best_cell: return True
        if min_choices == 0: return False

        r, c = best_cell
        for val in range(9):
            if best_allowed & (1 << val):
                mask = 1 << val
                grid[r, c] = val + 1
                rows[r] |= mask; cols[c] |= mask; boxes[get_box(r, c)] |= mask
                
                update_plot(grid) # Update the display
                
                if backtrack(): return True
                
                grid[r, c] = 0
                rows[r] &= ~mask; cols[c] &= ~mask; boxes[get_box(r, c)] &= ~mask
                update_plot(grid) # Update backtrack
        return False

    if backtrack():
        ax.set_title("Solved!", fontsize=14, color='green')
        clear_output(wait=True)
        display(fig)
    else:
        print("No solution found.")
    
    plt.close() # Prevent extra static plot at the end



solve_visual_notebook(board)